In [37]:
%matplotlib inline
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import keras, numpy as np
from keras.layers import Embedding, Dense, LSTM, GRU, Conv1D, Reshape
from keras.models import Sequential
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
import glob
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.preprocessing import text, sequence
from keras import utils
import pickle
import json
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools

print("You have TensorFlow version", tf.__version__)

You have TensorFlow version 1.7.0


In [3]:
files = glob.glob("./data/article_data*")
data = pd.concat([pd.read_csv(f, sep='|', engine='python', header=None) for f in files])
data.columns = ['article_id', 'body', 'topic']
data = data[~data.body.isnull()]
data.shape

(189295, 3)

In [4]:
# Split data into train and test
train_size = int(len(data) * .85)
print ("Train size: %d" % train_size)
print ("Test size: %d" % (len(data) - train_size))

Train size: 160900
Test size: 28395


In [5]:
train_body  = data['body'][:train_size]
train_topic = data['topic'][:train_size]

test_body = data['body'][train_size:]
test_topic = data['topic'][train_size:]

In [7]:
nb_words  = 10000
try:
    with open('model/tokenizer.pickle', 'rb') as handle:
        tokenize = pickle.load(handle)
except Exception:
    tokenize = text.Tokenizer(num_words=nb_words)
    tokenize.fit_on_texts(train_body) # only fit on train
    with open('model/tokenizer.pickle', 'wb') as handle:
        pickle.dump(tokenize, handle, protocol=pickle.HIGHEST_PROTOCOL)
x_train = tokenize.texts_to_matrix(train_body)
x_test = tokenize.texts_to_matrix(test_body)

In [8]:
topics = list(train_topic.unique())
ytrain_encoded = [topics.index(topic) for topic in train_topic] 
ytest_encoded = [topics.index(topic) for topic in test_topic]

y_train = keras.utils.to_categorical(ytrain_encoded)
y_test = keras.utils.to_categorical(ytest_encoded)
n_classes = len(topics)
print('classes: ', n_classes, 'less than 10 %', str(sum(train_topic.value_counts() <= 10)/train_topic.shape[0]))
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

classes:  282 less than 10 % 0.0001305158483530143
x_train shape: (160900, 20000)
x_test shape: (28395, 20000)
y_train shape: (160900, 282)
y_test shape: (28395, 282)


In [15]:
batch_size = 512
epochs = 15

word_index = tokenize.word_index


embedding_dim = embedding_vector_length = 64
sequence_length = max_seq_len = 1000

vocabulary_size = nb_words
filter_sizes = [3,4,5]
num_filters = 512
drop = 0.5




In [16]:
def dnn_model():
    
    global graph
    graph = tf.get_default_graph()
    print('building DNN model: ')
    model = Sequential()
    model.add(Dense(512, input_shape=(20000,)))
    model.add(Activation('relu'))
    model.add(Dropout(0.25))
    model.add(Dense(n_classes))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    
    print("model built: ", model.summary())
    return model

def cnn_model():
    print('building CNN model: ')
    from keras.layers import Input, Dense, Embedding, Conv2D, MaxPool2D
    from keras.layers import Reshape, Flatten, Dropout, Concatenate
    from keras.callbacks import ModelCheckpoint
    from keras.optimizers import Adam
    from keras.models import Model
    from sklearn.model_selection import train_test_split
    from data_helpers import load_data
    
#     model = Sequential()
#     model.add(Embedding(len(word_index)+1,
#                     embedding_vector_length, input_length=max_seq_len, embeddings_initializer='glorot_normal', 
#                         embeddings_regularizer=keras.regularizers.l1(0.01)))
    
#     model.add(Reshape((max_seq_len, embedding_vector_length,1)))

    inputs = Input(shape=(sequence_length,), dtype='int32')
    embedding = Embedding(input_dim=vocabulary_size, output_dim=embedding_dim, input_length=sequence_length)(inputs)
    reshape = Reshape((sequence_length,embedding_dim,1))(embedding)

    conv_0 = Conv2D(num_filters, kernel_size=(filter_sizes[0], embedding_dim), padding='valid', kernel_initializer='glorot_normal', activation='relu')(reshape)
    conv_1 = Conv2D(num_filters, kernel_size=(filter_sizes[1], embedding_dim), padding='valid', kernel_initializer='glorot_normal', activation='relu')(reshape)
    conv_2 = Conv2D(num_filters, kernel_size=(filter_sizes[2], embedding_dim), padding='valid', kernel_initializer='glorot_normal', activation='relu')(reshape)

    maxpool_0 = MaxPool2D(pool_size=(sequence_length - filter_sizes[0] + 1, 1), strides=(1,1), padding='valid')(conv_0)
    maxpool_1 = MaxPool2D(pool_size=(sequence_length - filter_sizes[1] + 1, 1), strides=(1,1), padding='valid')(conv_1)
    maxpool_2 = MaxPool2D(pool_size=(sequence_length - filter_sizes[2] + 1, 1), strides=(1,1), padding='valid')(conv_2)

    concatenated_tensor = Concatenate(axis=1)([maxpool_0, maxpool_1, maxpool_2])
    flatten = Flatten()(concatenated_tensor)
    dropout = Dropout(drop)(flatten)
    output = Dense(n_classes, activation='softmax')(dropout)

    model = Model(inputs=inputs, outputs=output)
    checkpoint = ModelCheckpoint('weights.{epoch:03d}-{val_acc:.4f}.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='auto')
    adam = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
    
    print("model built: ", model.summary())
    
    return model

In [17]:
model = dnn_model()
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

building DNN model: 
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 512)               10240512  
_________________________________________________________________
activation_5 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 282)               144666    
_________________________________________________________________
activation_6 (Activation)    (None, 282)               0         
Total params: 10,385,178
Trainable params: 10,385,178
Non-trainable params: 0
_________________________________________________________________
model built:  None
Train on 144810 samples, validate on 16090 samples
Epoch 1/15
144810/144810 [=============

In [18]:
# Evaluate the accuracy of our trained model
score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
print('Test score:', score[0])
print('Test accuracy:', score[1])

28395/28395 [==============================] - 4s 131us/step
Test score: 2.3592046725328046
Test accuracy: 0.38112343731932413


In [22]:
model.save('./model/dnn/topic_model.h5')

In [23]:
with open('./model/topics.txt', 'a') as f:
    f.write(json.dumps(list(train_topic)))

In [62]:
for i in range(15):
    prediction = model.predict(np.array([x_test[i]]))
    max_prob = np.argmax(prediction)
    predicted_label = topics[max_prob]
    print(test_body.iloc[i][:75], "... ")
    print('Actual label:' + test_topic.iloc[i])
    print("Predicted label: " + predicted_label )
    print("Confidence: " + str(np.round(prediction[:, max_prob][0] * 100, 2)) + "%\n")

Throwback Thursday: Earl Weaver Arrives in October, Is Promptly Ejected In  ... 
Actual label:baseball
Predicted label: baseball
Confidence: 59.11%

'The Littlest Memory,' Today's Comic by Leslie Stein Award winning cartooni ... 
Actual label:post-regular
Predicted label: post-regular
Confidence: 86.81%

An Ad Company Is Flying Surveillance Drones Over Los Angeles And there's no ... 
Actual label:privacy
Predicted label: motherboard show
Confidence: 49.33%

Animated Music Video Turns a Phosphorescent Garden into Rave Central Animat ... 
Actual label:dance
Predicted label: music videos
Confidence: 30.76%

Trevor Booker Beats Shot Clock With Ridiculous Backwards Shot Trevor Booker ... 
Actual label:basketball
Predicted label: basketball
Confidence: 61.48%

Dead Calm This is chapter 21 of Robert Young Pelton and Tim Freccia's spraw ... 
Actual label:violence
Predicted label: vice magazine
Confidence: 32.26%

Daily Horoscope: November 03, 2016 Daily horoscopes and astrology for Aquar ... 


In [48]:
prediction.shape

(1, 282)